In [1]:
# Misc. / Necessary Packages
import numpy as np
import pandas as pd
import os
import scipy
import scipy.io

import sys

# Bayesian inference libs
import pystan

# Plotting Libs
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

np.random.seed(1234)
%matplotlib inline

In [2]:
# first set up a simple hierarchical model of unpooled data
unpooled_data = """
data {
    int<lower=0> N; // number of channels
    int<lower=0> T; // number of time windows
    vector[T] y;
}

parameters {
  vector[85] a;
  real beta;
  real<lower=0,upper=100> sigma;
} 
transformed parameters {
  vector[N] y_hat;

  for (i in 1:N)
    y_hat[i] <- beta * x[i] + a[county[i]];
}
model {
  y ~ normal(y_hat, sigma);
}
"""

In [ ]:
unpooled_data = {'N': len(log_radon),
               'county': county+1, # Stan counts starting at 1
               'x': floor_measure,
               'y': log_radon}

unpooled_fit = pystan.stan(model_code=unpooled_model, data=unpooled_data, iter=1000, chains=2)

unpooled_estimates = pd.Series(unpooled_fit['a'].mean(0), index=mn_counties)
unpooled_se = pd.Series(unpooled_fit['a'].std(0), index=mn_counties)

order = unpooled_estimates.sort_values().index

plt.scatter(range(len(unpooled_estimates)), unpooled_estimates[order])
for i, m, se in zip(range(len(unpooled_estimates)), unpooled_estimates[order], unpooled_se[order]):
    plt.plot([i,i], [m-se, m+se], 'b-')
plt.xlim(-1,86); plt.ylim(-1,4)
plt.ylabel('Radon estimate');plt.xlabel('Ordered county');